In [1]:
from mpi import create_distinct_view, create_identity_view, link
import pandas as pd
import os
from utils.db import get_session
from utils.models import *

## Ingest
Bring .csv into the system

In [2]:
from ingest import load_file
os.listdir('assets/data')

['dws_wages.csv',
 'usbe_students.csv',
 'ushe_students.csv',
 'ustc_students.csv']

In [3]:
load_file('assets/data/dws_wages.csv', 'dws_wages')
load_file('assets/data/usbe_students.csv', 'usbe_students')
load_file('assets/data/ustc_students.csv', 'ustc_students')
load_file('assets/data/ushe_students.csv', 'ushe_students')

## Linking Steps

Process of Record Linkage

0. Load
1. Preprocess
2. Index
3. Compare
4. Classify
5. Evaluate
6. Update (Post MPI - Adding new information, new MPI, to MPI Master Record)

### Master Person Long
Visualize data architecture store of MPI raw records

In [4]:
# Visualize raw MPI data
#  V0.1  Data is seeded with ushe_students table
#  TODO: implement matched route and proper identification of unmatched

with get_session() as session:
    result = session.execute(
        'SELECT * FROM master_person_long LIMIT 5'
    ).fetchall()
    count_mpi = session.execute(
        'SELECT COUNT(DISTINCT(mpi)) FROM master_person_long'
    ).fetchone()
columns = ('mpi', 'field', 'value', 'score', 'guid')
pd.DataFrame(result, columns=columns)

,mpi,field,value,score,guid
0,4587782-4309556-16453842-9029444,ushe_student_id_pool,469629.0,1.0,1595745416251645427
1,4587782-4309556-16453842-9029444,last_name_pool,Steed,1.0,1595745416251645427
2,4587782-4309556-16453842-9029444,first_name_pool,Sakayla,1.0,1595745416251645427
3,4587782-4309556-16453842-9029444,middle_name_pool,Conder,1.0,1595745416251645427
4,4587782-4309556-16453842-9029444,birth_date_pool,6/20/2859,1.0,1595745416251645427


In [5]:
print('Total MPI in system: ', count_mpi[0])

Total MPI in system:  10000


## Load Data

In [6]:
source_tablename = 'ushe_students'

In [7]:
# Create a view of the data with mapped columns
raw, subset = create_distinct_view(source_tablename)
dview = subset.drop_duplicates()
# print(dview.head())

# Create a view from the MPI table with valid identity data
iview = create_identity_view(dview.columns.tolist())
# print(iview.head())

In [8]:
dview.head(1)

,ushe_student_id_pool,last_name_pool,first_name_pool,middle_name_pool,birth_date_pool,gender_pool,ssid_pool,ssn_pool,guid
0,469629.0,Steed,Sakayla,Conder,6/20/2859,m,0x23570dbc0x46ae1b78,375910347,1595745416251645427


In [9]:
iview.head(1)

mpi


Check here for potential for match.  If NO matching columns exist, mpis will need to be created for the dview.

In [10]:
# Check for match availability.  If not, halt process and create MPIs
from mpi.link import is_match_available
from mpi.update import generate_mpi

if not is_match_available(dview, iview):
    generate_mpi(dview)
    print('Match unavailable.  Generated MPIs for data view.')
    # Recreate a view from the MPI table with valid identity data
    iview = create_identity_view(dview.columns.tolist())
else:
    print('Match available.  Proceed with linking process.')

Match unavailable.  Generated MPIs for data view.


## Building record linkage and mpi classification

In [12]:
from mpi.link import clean_raw, match_dtype

### Preprocessing

Standardize data across data and identity views.

In [13]:
# Match Dtypes - Align data types prior to cleaning.
#    This helps the cleaner by segmenting string/object and numeric fields

# t2 identity table is a multi-index table.
#    If the table is empty, it will not have the value & score attributes
#    The if/else just checks whether or not the pivot of the mpi view yielded value and score fields.

if hasattr(iview, 'value'):
    ivalue = iview.value
else:
    ivalue = iview
    
if hasattr(iview, 'score'):
    iscore = iview.score

# Cast columns to matching datatypes for comparisons later on
source_data, id_data = match_dtype(dview, ivalue)  

# Clean data and re-index comparison.
subset = clean_raw(subset)
source_data = clean_raw(source_data)
id_data = clean_raw(id_data)
id_data = id_data.reset_index(level='mpi')

In [14]:
source_data.head(1)

,ushe_student_id_pool,last_name_pool,first_name_pool,middle_name_pool,birth_date_pool,gender_pool,ssid_pool,ssn_pool,guid
0,469629.0,steed,sakayla,conder,6/20/2859,m,0x23570dbc0x46ae1b78,375910347,1595745416251645427


In [15]:
id_data.head(1)

field,mpi,birth_date_pool,first_name_pool,gender_pool,guid,last_name_pool,middle_name_pool,ssid_pool,ssn_pool,ushe_student_id_pool
0,10000700-736429-12469678-3639982,9/26/2864,urijah,m,1595745416251645427,fortney,elizabeth kamalii wahine kai,0x379be1f90x6f37c3f2,450048739,511617.0


In [16]:
id_score.head(1)

field,birth_date_pool,first_name_pool,gender_pool,guid,last_name_pool,middle_name_pool,ssid_pool,ssn_pool,ushe_student_id_pool
mpi,,,,,,,,,
10000700-736429-12469678-3639982,1,1,1,1,1,1,1,1,1


## Indexing

Make record pairs - pair rows needing match to potential identity candidates.

Indexing serves two purposes:

1. Create the list of pairs to check (candidate link).  Example: row 1 from table 1 to row 199 from table 2.

2. Reduce the potential number of pairs to check (candidates).

In [17]:
from mpi.link import build_indexer, match_columns

In [18]:
# Create indexer on dataview
#    Indexer is a set of rules to generate 
#    candidate matches from data -> identities

source_matched, id_matched = match_columns(source_data, id_data)

indexer = build_indexer(source_matched)

# Check index algorithms (generated from data view columns)
indexer.algorithms

[<SortedNeighbourhood left_on='middle_name_pool', right_on='middle_name_pool'>,
 <SortedNeighbourhood left_on='first_name_pool', right_on='first_name_pool'>,
 <SortedNeighbourhood left_on='last_name_pool', right_on='last_name_pool'>,
 <Block left_on='ssn_pool', right_on='ssn_pool'>,
 <Block left_on='ssid_pool', right_on='ssid_pool'>,
 <Block left_on='ushe_student_id_pool', right_on='ushe_student_id_pool'>]

In [20]:
# Run indexer on dataview, identity view
candidates = indexer.index(source_matched, id_matched)

# Full indexing is a cross join of data and all possible identities.

# Demonstrating full indexing size:
print('Full Index Length: ', len(source_data) * len(id_data))

# Examine multi indices.  On the left is the data view index.  Right identity.
print('Algorithmic Index Length: ', len(candidates))

# Estimate Savings
print('Savings: ', (1- len(candidates)/(len(source_data) * len(id_data))) * 100)

# Preview indices:
for pair in candidates[0:5]:
    print(f'Data-row {pair[0]}', f'ID-row {pair[1]}')

Full Index Length:  100000000
Algorithmic Index Length:  94812
Savings:  99.905188
Data-row 0 ID-row 943
Data-row 0 ID-row 1258
Data-row 0 ID-row 2511
Data-row 0 ID-row 2855
Data-row 0 ID-row 3094


## Comparing

Indexing does not normally store the outcome of its findings.  Indexing algorithms are meant to be fast, can be error prone.  Algorithms can be tuned for string (many), numeric, and time/date fields.

The output of comparison is a clean feature matrix for the classifier to train/predict on.

In [21]:
from mpi.link import build_comparator

In [22]:
# Create comparator on dataview
#    Comparator is a set of algorithms for each feature to be compared.
#    These are genearlly much more expensive compared to indexing functions
cmp = build_comparator(source_matched)

# Check comparison algorithms and fields
cmp.features

[<Numeric 'ssn_pool'>,
 <Exact 'ssid_pool'>,
 <Numeric 'ushe_student_id_pool'>,
 <String 'middle_name_pool'>,
 <String 'first_name_pool'>,
 <String 'last_name_pool'>,
 <String 'gender_pool'>]

In [23]:
# Compute comparisons
#    Gives clean match dataset for classification
comparisons = cmp.compute(candidates, source_data, id_data)
comparisons.head()

ssn_pool  ssid_pool  ushe_student_id_pool  middle_name_pool  \
0 943        0.0          0                   0.0               0.0   
  1258       0.0          0                   0.0               1.0   
  2511       0.0          0                   0.0               0.0   
  2855       0.0          0                   0.0               1.0   
  3094       0.0          0                   0.0               0.0   

        first_name_pool  last_name_pool  gender_pool  
0 943               0.0             1.0          0.0  
  1258              0.0             0.0          0.0  
  2511              0.0             0.0          0.0  
  2855              0.0             0.0          1.0  
  3094              0.0             0.0          0.0

## Classification

Score candidates for match.  

#### Two approaches: Supervised vs Unsupervised
 * **Supervised** approach requires a training set.
 * **Unsupervised** does not require a training set and operates on only on the comparison table itself.

In [25]:
from mpi.link import estimate_true, build_classifier

# Get estimated true linkages for supervised model
links_true = estimate_true(comparisons)

# Create classifier
clf = build_classifier('logistic', comparisons, match_index=links_true)

# Check probabilities (score) of each comparison -- NOT IN USE IN THIS VERSION
predictions = clf.prob(comparison_vectors=comparisons)
predictions

0     943     0.000022
      1258    0.000030
      2511    0.000001
      2855    0.000110
      3094    0.000001
                ...   
9999  7599    0.000100
      8093    0.000027
      9166    0.000001
      9540    0.999436
      9685    0.000022
Length: 94812, dtype: float64

## Evaluate
Express classification quality and explore outliers

In [26]:
from recordlinkage import reduction_ratio
from recordlinkage import confusion_matrix

links_pred = clf.predict(comparison_vectors=comparisons)

rratio = reduction_ratio(links_pred, source_data)
cmatrix = confusion_matrix(links_true, links_pred, candidates)

In [27]:
# Review confusion matrix
#    TP-FN
#    |  |
#    FP-TN
print(cmatrix)

# Review reduction ratio
print(rratio)

[[ 9998     0]
 [    2 84812]]
0.9997999799979999


The confusion matrix may not be particularly useful here as generation of true links is prone to error. The reduction ratio is more sensitive than binary predictions in this case.

In [28]:
# Review findings
#   Interesting that the logistic predicted an MPI indices for each given an incomplete target list.

# Is the relationship 1,1?
split_list = lambda x: ([ix[0] for ix in x], [ix[1] for ix in x])
i1, i2 = split_list(links_pred)
len(list(set(i1))), len(list(set(i2)))

(10000, 10000)

## Update

Append matched MPIs and match score to data view and merge to original data.

In [29]:
from mpi.link import expand_match_to_raw

# Join data view (DISTINCT identities in source table), now containing matched and generated MPIs, to raw table.
#    This can be done a few ways.  Here, the data view (whose columns have been renamed and processed)
#    is joined to the original subset (whose columns were just renamed).  The subset is then indexed back 
#    unto the raw table so original column names and source formatting are preserved.


raw, matched, unmatched = expand_match_to_raw(raw.drop(['index'], axis=1), subset, source_data, id_data, links_pred)
raw.head(1)

,S_INST,S_YEAR,S_TERM,S_EXTRACT,S_ID,S_ID_FLAG,S_PREVIOUS_ID,S_LAST,S_FIRST,S_MIDDLE,...,S_TERM_ATT_CR,S_TERM_EARNED_CR,S_COLLEGE,S_MAJOR,S_COLLEGE2,S_MAJOR2,S_INAME,id,guid,mpi
0,1858,2018,3,E,469629.0,I,187277,Steed,Sakayla,Conder,...,4,126,Huntsman School of Business,Deaf Ed & Elementary Ed,School of the Arts,Network Systems,SNOW,7853,1595745416251645427,4587782-4309556-16453842-9029444


### Adding missing data, repeated data

When MPIs are generated for unmatched records, all data is used to populate the MPI table.  For matched MPIs, an update to missing information now needs to be performed to capture any new data.

What information is populated can have large performance and storage ramifications.
- If repeat data is kept, the initial identity view will need consolidation to reduce the number of possible indexes and comparisons.
- If repead data is NOT kept, significant information for background updates in match quality will be lost.

In [ ]:
# TODO

### De-Identification

Create de-identified table while match available in memory or as referenced temp table.

In [30]:
from assets.mapping import colmap
from utils.db import dataframe_to_db
from di import simple_di

dataframe_to_db(
    simple_di(raw), 
    tablename=source_tablename + '_di'
)

'ushe_students_di'